In [1]:
import torch
from torchinfo import summary
from transformers import AutoModelForCausalLM, AutoTokenizer

from tqdm.auto import tqdm

import pandas as pd

# Import necessary libraries
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import math

import random
from utils import save_data, load_data

## Inspecting the Phi-1 model

In [10]:
torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1", trust_remote_code=True)

C:\Users\leopu\AppData\Roaming\Python\Python39\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
summary(model)

Layer (type:depth-idx)                                  Param #
PhiForCausalLM                                          --
├─PhiModel: 1-1                                         --
│    └─Embedding: 2-1                                   104,857,600
│    └─Dropout: 2-2                                     --
│    └─ModuleList: 2-3                                  --
│    │    └─PhiDecoderLayer: 3-1                        50,354,176
│    │    └─PhiDecoderLayer: 3-2                        50,354,176
│    │    └─PhiDecoderLayer: 3-3                        50,354,176
│    │    └─PhiDecoderLayer: 3-4                        50,354,176
│    │    └─PhiDecoderLayer: 3-5                        50,354,176
│    │    └─PhiDecoderLayer: 3-6                        50,354,176
│    │    └─PhiDecoderLayer: 3-7                        50,354,176
│    │    └─PhiDecoderLayer: 3-8                        50,354,176
│    │    └─PhiDecoderLayer: 3-9                        50,354,176
│    │    └─PhiDecoderLayer: 

In [ ]:
inputs = tokenizer('''
Hello my name is             
''', return_tensors="pt", return_attention_mask=False)
print(inputs)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

## Replicating the textbook dataset creation

### Loading data

According to the original authors, they use three different source datasets:
- A filtered code-language dataset, which is a subset of The Stack and StackOverflow, obtained by
using a language model-based classifier (consisting of about 6B tokens).
- A synthetic textbook dataset consisting of <1B tokens of GPT-3.5 generated Python textbooks.
- A small synthetic exercises dataset consisting of ∼180M tokens of Python exercises and solutions.

In [2]:
from datasets import load_dataset

# Load the dataset in streaming mode
ds = load_dataset("bigcode/the-stack", data_dir="data/python", streaming=True, split="train")

# Initialize a counter
counter = 0

# Iterate over the dataset
dataset = {
    "sample": [],
    "label": [],
    "logprob": []
}

for sample in ds:
    dataset["sample"].append(sample["content"])
    counter += 1
    if counter >= 1000:
        break

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

In [ ]:
# Randomly check some examples from the dataset
import random 

random_samples = random.choices(dataset["sample"], k=3)
for i in random_samples: 
    print(i)
    print("-------")

### Label data with GPT-3.5-turbo

In [3]:
from openai import OpenAI
import os

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def gpt_labeling(sample: str, model_type: str="gpt-3.5-turbo-0125"): 
    response = client.chat.completions.create(
        model=model_type,
        messages=[
            {"role": "system", "content": """
                You are an AI assistant and your job is to classify. 
                Your job is to determine its educational value for a student whose goal is to learn basic coding concepts. 
            
                Here are the main points if an example is of a bad quality: 
                - Many samples are not self-contained, meaning that they depend on other modules or files that are
                external to the snippet, making them hard to understand without additional context.
                - Typical examples do not involve any meaningful computation, but rather consist of trivial or boil-
                erplate code, such as defining constants, setting parameters, or configuring GUI elements.
                - Samples that do contain algorithmic logic are often buried inside complex or poorly documented
                functions, making them difficult to follow or learn from.
                - The examples are skewed towards certain topics or use cases, resulting in an unbalanced distribution
                of coding concepts and skills across the dataset.
            
                If the educational value is high, return a 1. If the educational value is low, return a 0. 
                Return ONLY a number and nothing else. Otherwise I will NOT process your output!
            """},
            {"role": "user", "content": f"Code example: {sample[:10000]}"},
            {"role": "user", "content": "Classification: "}
        ],
        temperature=0.0,
        logprobs=True,
        logit_bias={15: 1, 16: 1},
        max_tokens=1, 
    )

    return response

In [5]:
for i in tqdm(range(len(dataset["sample"]))): 
    # Label data with GPT-3.5
    response = gpt_labeling(sample=dataset["sample"][i])

    # Get the label from the response
    label = response.choices[0].message.content
    logprobs = response.choices[0].logprobs.content[0].logprob

    # Add the label and prob to the dataset
    dataset["label"].append(int(label))
    dataset["logprob"].append(float(logprobs))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
training_df = pd.DataFrame(dataset)

In [13]:
# Check label distribution
training_df["label"].value_counts()

0    564
1    436
Name: label, dtype: int64

In [15]:
save_data(training_df, "./data/training-subset-labeled-1000.parquet")

### Training a random forest classifier

In [35]:
df = load_data("./data/training-subset-labeled.parquet")

X = df["sample"].tolist()
y = df["label"].tolist()

In [36]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("microsoft/codebert-base")

X_embedded = model.encode(X)

No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with MEAN pooling.


In [37]:
# import requests
# import os 

# hf_key = os.getenv("HF_API_KEY")

# API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/intfloat/e5-small-v2"
# headers = {"Authorization": f"Bearer {hf_key}"}

# def query(texts):
#     response = requests.post(API_URL, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
#     return response.json()
	
# X_embedded = query(texts=X)

In [38]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_embedded, y, test_size=0.2, random_state=42)

In [39]:
X_train[0]

array([-5.07316470e-01,  2.28404537e-01,  3.19959611e-01, -8.99160653e-02,
       -3.03865075e-01, -7.22756386e-01, -1.73169514e-03,  4.02829677e-01,
        3.07657182e-01,  4.90501344e-01, -2.62190342e-01,  8.52692842e-01,
       -2.67546445e-01, -3.12847883e-01,  8.49323392e-01, -2.10826129e-01,
        2.10718408e-01,  4.13056642e-01, -1.18505880e-02, -1.34410530e-01,
       -2.42161959e-01, -2.29697570e-01,  6.18695974e-01, -7.99583077e-01,
        3.46429884e-01,  4.40871119e-01, -6.44949675e-02,  7.53669143e-01,
       -5.93237162e-01,  9.20362294e-01, -2.09415555e-01,  2.70892143e-01,
        1.42405534e+00,  1.58426315e-01,  5.43486834e-01, -4.18687552e-01,
       -5.12024939e-01,  2.20933735e-01,  1.29838452e-01, -4.68540162e-01,
       -1.08978644e-01,  5.91604114e-01, -9.44795370e-01, -2.06896085e-02,
        4.49479342e-01,  3.86463046e-01,  5.64160347e-01, -2.75487095e-01,
        5.91123551e-02,  6.77372217e-01,  5.91932654e-01,  2.54811972e-01,
       -6.32566094e-01, -

In [40]:
y_train[:3]

[0, 0, 0]

In [41]:
from sklearn.linear_model import LogisticRegression

In [44]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)
print(y_pred)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.8


### Generating the synthetic textbook dataset

In [16]:
from openai import OpenAI
import os

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [28]:
def gpt_data_generation(topic: str, model_type: str="gpt-3.5-turbo-0125"): 
    response = client.chat.completions.create(
        model=model_type,
        messages=[
            {"role": "system", "content": """
                You are an expert on Python and an author of Python textbooks. 
                Your job is to create snippets of Python code with detailed English explanations. 
                The explanation should be at least five to eight sentences and be places above and below the code. 
                Ensure that all the code is of a very high quality and doesn't involve repetitive examples.
                Include comments in the generated code.
            """},
            {"role": "user", "content": f"The code and text should be about {topic}"},
            {"role": "user", "content": "Textbook snippet: "}
        ],
        temperature=0.4,
        max_tokens=512, 
    )

    return response.choices[0].message.content

In [29]:
topics = [
    "Introduction to Python: Basic syntax, variables, and data types",
    "Control Flow: Conditional statements (if, elif, else) and loops (for, while)",
    "Functions: Defining and calling functions, parameter passing",
    "Data Structures: Lists, tuples, dictionaries, sets",
    "File Handling: Reading from and writing to files",
    "Exception Handling: Handling errors and exceptions gracefully",
    "Object-Oriented Programming (OOP): Classes, objects, inheritance, polymorphism",
    "Modules and Packages: Importing and using external libraries",
    "String Manipulation: String methods, formatting, regular expressions",
    "Working with Dates and Times: Date objects, timedelta, formatting dates",
    "Input/Output: User input, output formatting",
    "List Comprehensions: Concise way to create lists",
    "Generators and Iterators: Iterable objects, yield statement",
    "Recursion: Functions calling themselves, solving problems recursively",
    "Functional Programming: Lambda functions, map, filter, reduce",
    "Debugging Techniques: Using print statements, debugging tools",
    "Testing: Writing and running tests using unittest or pytest",
    "Web Scraping: Extracting data from websites using libraries like BeautifulSoup",
    "GUI Programming: Creating graphical user interfaces with Tkinter or PyQt",
    "Data Visualization: Creating charts, graphs, and plots with libraries like Matplotlib or Seaborn",
    "NumPy: Introduction to numerical computing in Python",
    "Pandas: Data manipulation and analysis library for Python",
    "PyTorch: Deep learning framework for building and training neural networks"
]


In [33]:
datapoints = []
for i in tqdm(range(250)): 
    random_topic = random.choice(topics)

    # Generate a textbook snippet using GPT-3.5
    textbook_page = gpt_data_generation(topic=random_topic)
    datapoints.append(textbook_page)

  0%|          | 0/250 [00:00<?, ?it/s]

In [36]:
textbook_df = pd.DataFrame({"sample": datapoints})

In [39]:
save_data(textbook_df, "./data/synthetic-textbook-01.parquet")

### Concatinating the datasets

In [ ]:
sep_token = "<|endoftext|>"

## Building the model

In [3]:
import torch
from torch import nn

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")

In [7]:
# Assuming df is your DataFrame and "samples" is your column
def count_unique_words(df, column):
    # Split the strings into words, concatenate them and count the unique words
    unique_words = pd.Series(' '.join(df[column]).split()).nunique()
    return unique_words

# Call the function
unique_words = count_unique_words(df, "sample")
print(f"The column 'samples' contains {unique_words} unique words.")

The column 'samples' contains 23670 unique words.


In [14]:
# Hyperparams for phi-small
num_layers = 20 
hidden_dim = 1024
mlp_dim = 4096
num_heads = 16
attention_head_dim = 64

# Hyperparams Optimizer
learning_rate = 1e-3
weight_decay = 0.1
warmup_steps = 750
vocab_size = unique_words # set this

class PhiModel(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 hidden_dim: int = 1024,
                 num_heads: int = 16,
                 mlp_dim: int = 4096,
                 num_layers: int = 16,
                 dropout: float = 0.1,
                 activation: str="gelu"):
        super(PhiModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.dropout = nn.Dropout(dropout)

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=mlp_dim,
            dropout=dropout,
            activation=activation
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # MLP output layer
        self.output_layer = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, memory): 
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.transformer_decoder(x, memory)
        x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [15]:
phi_model = PhiModel(vocab_size=vocab_size)

summary(phi_model)

Layer (type:depth-idx)                                            Param #
PhiModel                                                          --
├─Embedding: 1-1                                                  24,238,080
├─Dropout: 1-2                                                    --
├─TransformerDecoder: 1-3                                         --
│    └─ModuleList: 2-1                                            --
│    │    └─TransformerDecoderLayer: 3-1                          16,796,672
│    │    └─TransformerDecoderLayer: 3-2                          16,796,672
│    │    └─TransformerDecoderLayer: 3-3                          16,796,672
│    │    └─TransformerDecoderLayer: 3-4                          16,796,672
│    │    └─TransformerDecoderLayer: 3-5                          16,796,672
│    │    └─TransformerDecoderLayer: 3-6                          16,796,672
│    │    └─TransformerDecoderLayer: 3-7                          16,796,672
│    │    └─TransformerDecoderLaye

## Training the model 